# IBM Data Science Capstone Project

## || Part 4 : Interactive Map with Folium

In [1]:
import folium
import wget
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
import pandas as pd

In [2]:
# Using the ready dataset from the previous activities
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
df=pd.read_csv(spacex_csv_file)

100% [................................................................................] 7710 / 7710

In [3]:
# Subset the relevant subcolumn for GEOMAP VISUALIZATION purposes
df = df[['Launch Site', 'Lat', 'Long', 'class']]

# It will grouping, and choose the FIRST ROW on the same group
launchSiteDf = df.groupby(['Launch Site'], as_index = False).first()
launchSiteDf = launchSiteDf[['Launch Site', 'Lat', 'Long']]
launchSiteDf

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [4]:
# Next, we will visualizing the given coordinates by some following steps

# FOR EXAMPLE : Create a folium map object, with an initial center location to be NASA center at TEXAS
nasaCoordinate = [29.559684888503615, -95.0830971930759]
initialMap = folium.Map(location=nasaCoordinate, zoom_start=10)

In [5]:
# After that, we will ad highlighted circle area with a text label on the specific coordinate

# Create blue circle at NASA Center's coordinate with a popup label name
circle = folium.Circle(nasaCoordinate, radius = 1000, color = '#d35400', fill = True).add_child(folium.Popup("NASA Johnson Space Center"))
marker = folium.map.Marker(
    nasaCoordinate,
    icon = DivIcon(
        icon_size = (20,20),
        icon_anchor = (0,0),
        html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>'%'NASA JSC',
    )
)
initialMap.add_child(circle)
initialMap.add_child(marker)

1. Marking all Launch Sites Area on Map

In [6]:
# Next, creating the geo-map function for each SpaceX Launch Area

SitesMap = folium.Map(nasaCoordinate, zoom_start = 3)

for index, row in launchSiteDf.iterrows():
  areaCoordinate = [row['Lat'], row['Long']]
  label = row['Launch Site']
  circle = folium.Circle(areaCoordinate, radius = 1000, color = "#d35400", fill = True).add_child(folium.Popup(label))
  marker = folium.map.Marker(
    areaCoordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % label,
        )
    )
  SitesMap.add_child(circle)
  SitesMap.add_child(marker)

SitesMap

2. Mark the Success/Failed Records for Each Launch Sites

In [7]:
marker_cluster = MarkerCluster()

In [8]:
def AssignColor(launchOutcomes):
  if launchOutcomes == 1:
    return "green"
  else:
    return "red"

df['MarkerColor'] = df['class'].apply(AssignColor) # df['MarkerColor'] = AssignColor(df['MarkerColor'])
df.tail(10)

,Launch Site,Lat,Long,class,MarkerColor
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [9]:
SitesMap.add_child(marker_cluster)

for index, observation in df.iterrows():
  areaCoordinate = [observation['Lat'], observation['Long']]
  marker = folium.Marker(
      areaCoordinate,
      icon = folium.Icon(
          color = observation['MarkerColor'],
          icon = 'circle'
      )
  )
  marker_cluster.add_child(marker)

SitesMap

3. Calculating Distance between A Launch Site to Its Proximities

In [10]:
# Add MousePosition on map, to get coordinate for a mouse over a point on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mousePosition = MousePosition(
    position = 'topright',
    separator = ' Long: ',
    empty_string = 'NaN',
    lng_first = False,
    num_digits = 20,
    prefix = 'Lat:',
    lat_formatter = formatter,
    lng_formatter = formatter,
)

SitesMap.add_child(mousePosition)
SitesMap

In [11]:
# Create a Function to calculate distance between two points on the map based on coordinates

from math import sin, cos, sqrt, atan2, radians

def DistanceCalculation(lat_1, long_1, lat_2, long_2):
  R = 6373.0 # The approximation of the earth radius (kilometers)
  lat_1 = radians(lat_1)
  long_1 = radians(long_1)
  lat_2 = radians(lat_2)
  long_2 = radians(long_2)

  distanceLong = long_2 - long_1
  distanceLat = lat_2 - lat_1

  a = sin(distanceLat / 2)**2 + cos(lat_1) * cos(lat_2) * sin(distanceLong / 2)**2
  c = 2 * atan2(sqrt(a), sqrt(1 - a))

  distance = R * c
  return distance

In [12]:
# Calculating the distance

for index, observation in df.iterrows():
  distanceCoastline = DistanceCalculation(28.56383, -80.56802, observation['Lat'], observation['Long'])
  df['DistanceCoastline'] = distanceCoastline

df.head(5)

,Launch Site,Lat,Long,class,MarkerColor,DistanceCoastline
0,CCAFS LC-40,28.562302,-80.577356,0,red,0.862567
1,CCAFS LC-40,28.562302,-80.577356,0,red,0.862567
2,CCAFS LC-40,28.562302,-80.577356,0,red,0.862567
3,CCAFS LC-40,28.562302,-80.577356,0,red,0.862567
4,CCAFS LC-40,28.562302,-80.577356,0,red,0.862567


In [13]:
# Add to the maps with the line
for index, observation in df.iterrows():
  distance = "{:.2f} KM".format(float(df['DistanceCoastline'].values[-1]))
  lines = folium.PolyLine(locations = [[28.56383, -80.56802], [observation['Lat'], observation['Long']]], weight = 1).add_child(folium.Popup(distance))
  markerDistance = folium.Marker(
      [observation['Lat'], observation['Long']],
      icon = DivIcon(
          icon_size=(20,20),
          icon_anchor=(-20,0),
          #html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:.2f} KM".format(observation['DistanceCoastline']),
          html = f'<div style="font-size: 16px; color:#d35400; margin-left: 10px; white-space: nowrap;"><b>{distance}</b></div>',
      )
  )
SitesMap.add_child(markerDistance)
SitesMap.add_child(lines)

SitesMap

### Created By : ALEXANDER RAFAEL
### 29 March 2025